In [4]:
import os
import datetime
import numpy as np
import pandas as pd
from os.path import dirname, abspath
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from data_processor import series_to_supervised,evaluate
from data_processor import series_to_supervised,evaluate
from sklearn.svm import SVR

In [19]:
from keras import Sequential
from keras.layers import Dense,Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.
/Users/mayuan/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/mayuan/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/mayuan/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/mayuan/anaconda3/lib/python3.6/site-packages/t

In [8]:
def CF(power, NWP):
    cloud = pd.DataFrame(NWP.cloud_amount)
    cloud['cloud_diff'] = cloud.diff(1)
    cloud = cloud.fillna(0)

    cloud['fluctuate'] = [0 if ((cloud.cloud_amount[i]>95 or cloud.cloud_amount[i]<5) and abs(cloud.cloud_diff[i])<1)
                          else 1
                          for i in range(len(cloud))]

    cloud = cloud.loc[power.index]

    index_stable = [x for x in cloud[cloud.fluctuate == 0].index if x < datetime.datetime(2019,1,1)]
    index_fluc = [x for x in cloud[cloud.fluctuate == 1].index if x < datetime.datetime(2019,1,1)]

    power_stable = power.loc[index_stable]
    power_fluc = power.loc[index_fluc]

    return index_stable,index_fluc

In [47]:
def sample_generator(power, index_stable, train_len, test_len):
    # 返回归一化后的两个分类下的输入输出
    train_len = train_len
    test_len = test_len

    # normalizaion
    index = power.index
    scaler = MinMaxScaler(feature_range=(0, 1))
    power_normalized = pd.DataFrame(scaler.fit_transform(power))
    power_normalized['index'] = index
    power_normalized = power_normalized.set_index('index')

    power_supervised = series_to_supervised(power_normalized, 48, 16)

    index_of_inputs = list(range(16)) + list(range(0, -8, -1))
    inputs = power_supervised.iloc[:, :48]
    inputs = inputs.iloc[:,index_of_inputs]
    outputs = power_supervised.iloc[:, 48:]

    for i in range(16):
        names['output_%s' % str(i + 1)] = outputs.iloc[:, i]

    outputset = []
    for i in range(16):
        names['output_stable_%s' % str(i + 1)] = names['output_%s' % str(i + 1)].loc[
            list(map(lambda x: x + datetime.timedelta(minutes=15 * i), index_stable))]
        names['output_stable_%s' % str(i + 1)] = names['output_stable_%s' % str(i + 1)].dropna()
        outputset.append(names['output_stable_%s' % str(i + 1)])

    inputset = []
    for i in range(16):
        names['input_stable_%s' % str(i + 1)] = inputs.loc[names['output_stable_%s' % str(i + 1)].index]
        inputset.append(names['input_stable_%s' % str(i + 1)])

    dataset = []
    for i in range(len(inputset)):
        # train_x_set,train_y_set,test_x_set,test_y_set
        if (train_len+test_len) == 0:
            dataset.append([inputset[i][:train_len], outputset[i][:train_len], inputset[i][train_len:], outputset[i][train_len:]])
        else:
            dataset.append([inputset[i][:train_len], outputset[i][:train_len], inputset[i][train_len:(train_len + test_len)], outputset[i][:(train_len + test_len)]])

    return dataset, scaler

In [34]:
def ANN_sp(dataset, scaler):
    # 下一步集成化，ANN的输入参数为训练集和测试集

    # parameters
    input_len = dataset[0][0].shape[1]
    output_step = 1
    batch_size = 512
    epochs = 100

    names = locals()

    for (i,[train_x,train_y,test_x,test_y]) in enumerate(dataset):

        # design network
        model = Sequential()

        model.add(Dense(int(input_len / 2), input_dim=input_len, activation='relu'))
        model.add(Dropout(0.2))

        model.add(Dense(output_step, activation='relu'))

        adam = Adam(lr=0.001)

        model.compile(loss='mse', optimizer='adam')

        history = model.fit(train_x, train_y,
                            batch_size=batch_size,
                            epochs=epochs,
                            validation_split=0.1,
                            callbacks=[
                                TensorBoard(log_dir='/tmp/tensorboard', write_graph=True),
                                EarlyStopping(monitor='val_loss', patience=5, mode='auto')
                            ]
                            )

        # make a prediction
        names['y_hat_%s' % str(i + 1)] = model.predict(test_x)
        names['inv_yhat_%s' % str(i + 1)] = scaler.inverse_transform(names['y_hat_%s' % str(i + 1)])

        test_y = pd.DataFrame(test_y)
        names['inv_y_%s' % str(i + 1)] = scaler.inverse_transform(test_y)
        names['rmse_%s' % str(i + 1)] = evaluate(names['inv_y_%s' % str(i + 1)], names['inv_yhat_%s' % str(i + 1)])
        
    y_hat = []
    for j in range(16):
        y_hat.append(names['inv_yhat_%s' % str(j + 1)])
    y_hat = pd.DataFrame(np.array(y_hat).reshape(16, -1).T)

    rmse = []
    for j in range(16):
        rmse.append(names['rmse_%s' % str(j + 1)])
    rmse = np.array(rmse).reshape(-1)

    return y_hat, rmse

In [2]:
def SVR_sp(dataset, scaler):
    # 下一步集成化，ANN的输入参数为训练集和测试集

    # parameters
    input_len = dataset[0][0].shape[1]
    output_step = 1
    batch_size = 512
    epochs = 100

    names = locals()

    for (i, [train_x, train_y, test_x, test_y]) in enumerate(dataset):
        clf = SVR(kernel='rbf',
                  gamma='auto'
                  )

        clf.fit(train_x, train_y)

        # make a prediction
        names['y_hat_%s' % str(i + 1)] = clf.predict(test_x)
        names['inv_yhat_%s' % str(i + 1)] = scaler.inverse_transform(names['y_hat_%s' % str(i + 1)].reshape(-1,1))

        test_y = pd.DataFrame(test_y)
        names['inv_y_%s' % str(i + 1)] = scaler.inverse_transform(test_y)
        names['rmse_%s' % str(i + 1)] = evaluate(names['inv_y_%s' % str(i + 1)], names['inv_yhat_%s' % str(i + 1)])

    y_hat = []
    for j in range(16):
        y_hat.append(names['inv_yhat_%s' % str(j + 1)])
    y_hat = pd.DataFrame(np.array(y_hat).reshape(16, -1).T)

    rmse = []
    for j in range(16):
        rmse.append(names['rmse_%s' % str(j + 1)])
    rmse = np.array(rmse).reshape(-1)

    return y_hat, rmse

In [6]:
names = locals()
# load power
dir_power = 'power_detrended/'

name_index_power = pd.Series(os.listdir(dir_power)).map(lambda x: x[:-4])

for i in name_index_power:
    filename = dir_power + str(i) + '.csv'
    names['data_%s' % i] = pd.read_csv(filename, index_col=3, parse_dates=True)
    names['power_%s' % i] = pd.DataFrame(names['data_%s' % i]['power_with_trend'])
    names['power_supervised_%s' % i] = series_to_supervised(names['power_%s' % i], 48, 16)

# load NWP
dir_NWP = 'nwp_refill/'

name_index_NWP = list(os.listdir(dir_NWP))
name_index_NWP.remove('.DS_Store')
name_index_NWP = pd.Series(name_index_NWP).map(lambda x: x[3:-4])

for i in name_index_NWP:
    filename = dir_NWP + '/CN0' + str(i) + '.csv'
    names['NWP_%s' % i] = pd.read_csv(filename, index_col=0, parse_dates=True)

In [9]:
index_stable, index_fluc = CF(power_66, NWP_016)

In [11]:
# index_stable

In [48]:
dataset, scaler = sample_generator(power_66, index_stable, -1000, 1000)

/Users/mayuan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike


In [30]:
# dataset[0][0]

### ANN

In [51]:
y_hat, rmse = ANN_sp(dataset, scaler)

Train on 16809 samples, validate on 1868 samples
Epoch 1/100
16809/16809 [==============================] - 1s 67us/step - loss: 0.1598 - val_loss: 0.1101
Epoch 2/100
16809/16809 [==============================] - 0s 8us/step - loss: 0.0841 - val_loss: 0.0494
Epoch 3/100
16809/16809 [==============================] - 0s 7us/step - loss: 0.0530 - val_loss: 0.0261
Epoch 4/100
16809/16809 [==============================] - 0s 8us/step - loss: 0.0399 - val_loss: 0.0205
Epoch 5/100
16809/16809 [==============================] - 0s 8us/step - loss: 0.0320 - val_loss: 0.0179
Epoch 6/100
16809/16809 [==============================] - 0s 8us/step - loss: 0.0299 - val_loss: 0.0156
Epoch 7/100
16809/16809 [==============================] - 0s 9us/step - loss: 0.0262 - val_loss: 0.0139
Epoch 8/100
16809/16809 [==============================] - 0s 7us/step - loss: 0.0251 - val_loss: 0.0125
Epoch 9/100
16809/16809 [==============================] - 0s 6us/step - loss: 0.0235 - val_loss: 0.0116
Epoch

Train on 16455 samples, validate on 1829 samples
Epoch 1/100
16455/16455 [==============================] - 1s 67us/step - loss: 0.2084 - val_loss: 0.1494
Epoch 2/100
16455/16455 [==============================] - 0s 7us/step - loss: 0.1322 - val_loss: 0.0888
Epoch 3/100
16455/16455 [==============================] - 0s 7us/step - loss: 0.0936 - val_loss: 0.0605
Epoch 4/100
16455/16455 [==============================] - 0s 6us/step - loss: 0.0772 - val_loss: 0.0453
Epoch 5/100
16455/16455 [==============================] - 0s 6us/step - loss: 0.0667 - val_loss: 0.0327
Epoch 6/100
16455/16455 [==============================] - 0s 6us/step - loss: 0.0619 - val_loss: 0.0286
Epoch 7/100
16455/16455 [==============================] - 0s 7us/step - loss: 0.0539 - val_loss: 0.0223
Epoch 8/100
16455/16455 [==============================] - 0s 6us/step - loss: 0.0448 - val_loss: 0.0191
Epoch 9/100
16455/16455 [==============================] - 0s 6us/step - loss: 0.0364 - val_loss: 0.0171
Epoch

16100/16100 [==============================] - 0s 6us/step - loss: 0.0208 - val_loss: 0.0143
Epoch 21/100
16100/16100 [==============================] - 0s 6us/step - loss: 0.0207 - val_loss: 0.0143
Epoch 22/100
16100/16100 [==============================] - 0s 6us/step - loss: 0.0206 - val_loss: 0.0147
Epoch 23/100
16100/16100 [==============================] - 0s 6us/step - loss: 0.0204 - val_loss: 0.0141
Epoch 24/100
16100/16100 [==============================] - 0s 6us/step - loss: 0.0203 - val_loss: 0.0140
Epoch 25/100
16100/16100 [==============================] - 0s 6us/step - loss: 0.0205 - val_loss: 0.0146
Epoch 26/100
16100/16100 [==============================] - 0s 6us/step - loss: 0.0200 - val_loss: 0.0145
Epoch 27/100
16100/16100 [==============================] - 0s 6us/step - loss: 0.0201 - val_loss: 0.0142
Epoch 28/100
16100/16100 [==============================] - 0s 6us/step - loss: 0.0199 - val_loss: 0.0140
Epoch 29/100
16100/16100 [==============================] -

15039/15039 [==============================] - 1s 93us/step - loss: 0.1177 - val_loss: 0.0586
Epoch 2/100
15039/15039 [==============================] - 0s 7us/step - loss: 0.0675 - val_loss: 0.0450
Epoch 3/100
15039/15039 [==============================] - 0s 7us/step - loss: 0.0600 - val_loss: 0.0378
Epoch 4/100
15039/15039 [==============================] - 0s 6us/step - loss: 0.0534 - val_loss: 0.0324
Epoch 5/100
15039/15039 [==============================] - 0s 6us/step - loss: 0.0503 - val_loss: 0.0315
Epoch 6/100
15039/15039 [==============================] - 0s 7us/step - loss: 0.0477 - val_loss: 0.0298
Epoch 7/100
15039/15039 [==============================] - 0s 6us/step - loss: 0.0458 - val_loss: 0.0298
Epoch 8/100
15039/15039 [==============================] - 0s 6us/step - loss: 0.0441 - val_loss: 0.0288
Epoch 9/100
15039/15039 [==============================] - 0s 7us/step - loss: 0.0434 - val_loss: 0.0283
Epoch 10/100
15039/15039 [==============================] - 0s 7us

14683/14683 [==============================] - 0s 8us/step - loss: 0.0306 - val_loss: 0.0236
Epoch 34/100
14683/14683 [==============================] - 0s 7us/step - loss: 0.0305 - val_loss: 0.0242
Epoch 35/100
14683/14683 [==============================] - 0s 8us/step - loss: 0.0303 - val_loss: 0.0238
Epoch 36/100
14683/14683 [==============================] - 0s 6us/step - loss: 0.0302 - val_loss: 0.0240
Epoch 37/100
14683/14683 [==============================] - 0s 6us/step - loss: 0.0295 - val_loss: 0.0236
Epoch 38/100
14683/14683 [==============================] - 0s 7us/step - loss: 0.0297 - val_loss: 0.0243
Epoch 39/100
14683/14683 [==============================] - 0s 6us/step - loss: 0.0293 - val_loss: 0.0234
Epoch 40/100
14683/14683 [==============================] - 0s 7us/step - loss: 0.0295 - val_loss: 0.0234
Epoch 41/100
14683/14683 [==============================] - 0s 6us/step - loss: 0.0296 - val_loss: 0.0234
Epoch 42/100
14683/14683 [==============================] -

13973/13973 [==============================] - 0s 6us/step - loss: 0.0456 - val_loss: 0.0323
Epoch 14/100
13973/13973 [==============================] - 0s 7us/step - loss: 0.0444 - val_loss: 0.0312
Epoch 15/100
13973/13973 [==============================] - 0s 7us/step - loss: 0.0435 - val_loss: 0.0305
Epoch 16/100
13973/13973 [==============================] - 0s 7us/step - loss: 0.0425 - val_loss: 0.0298
Epoch 17/100
13973/13973 [==============================] - 0s 6us/step - loss: 0.0421 - val_loss: 0.0294
Epoch 18/100
13973/13973 [==============================] - 0s 7us/step - loss: 0.0421 - val_loss: 0.0294
Epoch 19/100
13973/13973 [==============================] - 0s 6us/step - loss: 0.0410 - val_loss: 0.0292
Epoch 20/100
13973/13973 [==============================] - 0s 6us/step - loss: 0.0403 - val_loss: 0.0283
Epoch 21/100
13973/13973 [==============================] - 0s 6us/step - loss: 0.0402 - val_loss: 0.0279
Epoch 22/100
13973/13973 [==============================] -

13619/13619 [==============================] - 0s 6us/step - loss: 0.0337 - val_loss: 0.0231
Epoch 43/100
13619/13619 [==============================] - 0s 6us/step - loss: 0.0338 - val_loss: 0.0230
Epoch 44/100
13619/13619 [==============================] - 0s 6us/step - loss: 0.0336 - val_loss: 0.0234
Epoch 45/100
13619/13619 [==============================] - 0s 6us/step - loss: 0.0335 - val_loss: 0.0228
Epoch 46/100
13619/13619 [==============================] - 0s 6us/step - loss: 0.0332 - val_loss: 0.0233
Epoch 47/100
13619/13619 [==============================] - 0s 7us/step - loss: 0.0335 - val_loss: 0.0232
Epoch 48/100
13619/13619 [==============================] - 0s 6us/step - loss: 0.0333 - val_loss: 0.0232
Epoch 49/100
13619/13619 [==============================] - 0s 6us/step - loss: 0.0334 - val_loss: 0.0231
Epoch 50/100
13619/13619 [==============================] - 0s 6us/step - loss: 0.0334 - val_loss: 0.0227
Epoch 51/100
13619/13619 [==============================] -

12905/12905 [==============================] - 0s 7us/step - loss: 0.0624 - val_loss: 0.0450
Epoch 12/100
12905/12905 [==============================] - 0s 7us/step - loss: 0.0616 - val_loss: 0.0425
Epoch 13/100
12905/12905 [==============================] - 0s 6us/step - loss: 0.0584 - val_loss: 0.0407
Epoch 14/100
12905/12905 [==============================] - 0s 7us/step - loss: 0.0563 - val_loss: 0.0388
Epoch 15/100
12905/12905 [==============================] - 0s 7us/step - loss: 0.0551 - val_loss: 0.0369
Epoch 16/100
12905/12905 [==============================] - 0s 7us/step - loss: 0.0536 - val_loss: 0.0351
Epoch 17/100
12905/12905 [==============================] - 0s 6us/step - loss: 0.0529 - val_loss: 0.0350
Epoch 18/100
12905/12905 [==============================] - 0s 7us/step - loss: 0.0513 - val_loss: 0.0332
Epoch 19/100
12905/12905 [==============================] - 0s 7us/step - loss: 0.0492 - val_loss: 0.0315
Epoch 20/100
12905/12905 [==============================] -

12193/12193 [==============================] - 0s 7us/step - loss: 0.0517 - val_loss: 0.0429
Epoch 5/100
12193/12193 [==============================] - 0s 6us/step - loss: 0.0465 - val_loss: 0.0393
Epoch 6/100
12193/12193 [==============================] - 0s 7us/step - loss: 0.0445 - val_loss: 0.0371
Epoch 7/100
12193/12193 [==============================] - 0s 7us/step - loss: 0.0422 - val_loss: 0.0349
Epoch 8/100
12193/12193 [==============================] - 0s 6us/step - loss: 0.0402 - val_loss: 0.0328
Epoch 9/100
12193/12193 [==============================] - 0s 6us/step - loss: 0.0390 - val_loss: 0.0309
Epoch 10/100
12193/12193 [==============================] - 0s 6us/step - loss: 0.0387 - val_loss: 0.0301
Epoch 11/100
12193/12193 [==============================] - 0s 6us/step - loss: 0.0378 - val_loss: 0.0296
Epoch 12/100
12193/12193 [==============================] - 0s 6us/step - loss: 0.0372 - val_loss: 0.0291
Epoch 13/100
12193/12193 [==============================] - 0s 6

11484/11484 [==============================] - 0s 7us/step - loss: 0.0307 - val_loss: 0.0270
Epoch 20/100
11484/11484 [==============================] - 0s 7us/step - loss: 0.0305 - val_loss: 0.0272
Epoch 21/100
11484/11484 [==============================] - 0s 7us/step - loss: 0.0300 - val_loss: 0.0266
Epoch 22/100
11484/11484 [==============================] - 0s 7us/step - loss: 0.0303 - val_loss: 0.0276
Epoch 23/100
11484/11484 [==============================] - 0s 7us/step - loss: 0.0300 - val_loss: 0.0269
Epoch 24/100
11484/11484 [==============================] - 0s 7us/step - loss: 0.0299 - val_loss: 0.0265
Epoch 25/100
11484/11484 [==============================] - 0s 7us/step - loss: 0.0299 - val_loss: 0.0266
Epoch 26/100
11484/11484 [==============================] - 0s 7us/step - loss: 0.0294 - val_loss: 0.0262
Epoch 27/100
11484/11484 [==============================] - 0s 7us/step - loss: 0.0291 - val_loss: 0.0260
Epoch 28/100
11484/11484 [==============================] -

In [32]:
rmse

array([0.88823153, 1.37750551, 1.70096192, 1.97462078, 2.37441264,
       2.59405676, 2.53595393, 2.76802686, 2.85895222, 2.91424532,
       2.98003151, 2.72162693, 2.52609441, 2.68812899, 2.54815012,
       2.55245768])

In [33]:
1-rmse/20

array([0.95558842, 0.93112472, 0.9149519 , 0.90126896, 0.88127937,
       0.87029716, 0.8732023 , 0.86159866, 0.85705239, 0.85428773,
       0.85099842, 0.86391865, 0.87369528, 0.86559355, 0.87259249,
       0.87237712])

### SVR

In [49]:
y_hat_svr, rmse_svr = SVR_sp(dataset, scaler)

In [46]:
1-rmse/20

array([0.95558842, 0.93112472, 0.9149519 , 0.90126896, 0.88127937,
       0.87029716, 0.8732023 , 0.86159866, 0.85705239, 0.85428773,
       0.85099842, 0.86391865, 0.87369528, 0.86559355, 0.87259249,
       0.87237712])

In [50]:
1-rmse/20

array([0.95558842, 0.93112472, 0.9149519 , 0.90126896, 0.88127937,
       0.87029716, 0.8732023 , 0.86159866, 0.85705239, 0.85428773,
       0.85099842, 0.86391865, 0.87369528, 0.86559355, 0.87259249,
       0.87237712])

In [ ]:
### 